In [20]:
import glob
import pandas as pd

path = 'homepage_csvs/test/'
#path = 'homepage_csvs/chaturbate/'

csv_files = glob.glob(path + '*.csv')

counter = 0
sessions = []
last_df = None

for filename in sorted(csv_files):
    
    df = pd.read_csv(filename)
    #print(df)
    
    if counter > 0:
        print('hi')
        
    last_df = df

    print(counter)
    counter += 1


#df = [pd.read_csv(filename) for filename in csv_files]
#df


0
hi
1
hi
2
hi
3
hi
4
hi
5
hi
6
